In [ ]:
!pip install --upgrade langchain openai  -q
!pip install sentence_transformers -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils

In [7]:
import os
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import WebBaseLoader,OnlinePDFLoader
import streamlit as st
import tiktoken
from bs4 import BeautifulSoup
import re
import requests
from urllib.parse import urljoin

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

In [3]:
urls = ['https://au.edu.pk/']

In [5]:
def is_admission_link(link):
    keyword_pattern = re.compile(r'Admission|admissions|Faculties', re.IGNORECASE)
    return bool(keyword_pattern.search(link))
def extract_links(urls):
    all_links = []
    for link in urls:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, "html.parser")
        base_url = response.url  # Get the base URL from the response object
        links = []
        for a_tag in soup.find_all("a"):
            href = a_tag.get("href")
            if href:
                absolute_link = urljoin(base_url, href)  # Convert relative link to absolute link
                if is_admission_link(absolute_link):
                    links.append(absolute_link)
        all_links.extend(links)
    print(len(all_links))
    return all_links
all_links = extract_links(urls)

59


In [6]:
loader = WebBaseLoader(all_links)
scrape_data = loader.load()
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts_from_links = text_splitter.split_documents(scrape_data)
for document in texts_from_links:
    document.page_content = document.page_content.replace('\n', ' ')
print('Links Data done')

Links Data done


In [8]:
loader = OnlinePDFLoader("https://au.edu.pk/AU_Documents/Details_of_Dept_Program_Representatives_for_Admissions%20Fall_2022.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts_from_pdf_link = text_splitter.split_documents(data)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abd_b\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\abd_b\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [9]:
text = texts_from_links+texts_from_pdf_link

loader = WebBaseLoader(all_links)
scrape_data = loader.load()
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts_from_links = text_splitter.split_documents(scrape_data)
for document in texts_from_links:
    document.page_content = document.page_content.replace('\n', ' ')
print('Links Data done')

In [ ]:
db_links = FAISS.from_documents(text, embeddings)
db_links.save_local('links_db')
print('Embeddings created and saved successfully.')

In [11]:
db_links = FAISS.from_documents(text, embeddings)

In [12]:
db_links.save_local('links_db') 

Embeddings created and saved successfully.


In [13]:
print('Embeddings created and saved successfully.')

Embeddings created and saved successfully.
